In [1]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')

import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

import os
from datetime import datetime

from ff_app.execution import run_data_pull, update_google_sheet
from ff_app import data_prep

10:49:12 INFO:NumExpr defaulting to 8 threads.


In [2]:
week = 5

short_df = run_data_pull(week=week)
# short_df, all_df = run_data_pull(week=week, return_all_games=True)

print(short_df.shape)

10:49:12 DEBUG:Starting new HTTPS connection (1): api.the-odds-api.com:443
10:49:12 DEBUG:https://api.the-odds-api.com:443 "GET /v4/sports/americanfootball_ncaaf/odds?markets=spreads%2Ctotals&regions=us&oddsFormat=american&dateFormat=iso&api_key=ce31cd809f1458374e07be4035c3d210 HTTP/1.1" 200 15517
10:49:12 INFO:Pulled 56 total games for week 5 in 2023
10:49:12 INFO:Picks sheet data has shape (56, 11)
10:49:12 INFO:Saving game data to disk at '/Users/hoke/dev/football/game_lists/2023/week5.csv'


(56, 11)


In [3]:
short_df.head()

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Description,Spread,Total,Implied Score
0,5,09/28 07:30 PM (Thu),N,Western Kentucky Hilltoppers,Middle Tennessee Blue Raiders,Western Kentucky Hilltoppers,Middle Tennessee Blue Raiders,WKU vs MTSU,-5.5,60.0,32 - 27
1,5,09/28 07:30 PM (Thu),N,Tulsa Golden Hurricane,Temple Owls,Tulsa Golden Hurricane,Temple Owls,TUL vs TEM,-3.0,55.5,29 - 26
2,5,09/28 08:00 PM (Thu),N,Sam Houston State Bearkats,Jacksonville State Gamecocks,Jacksonville State Gamecocks,Sam Houston State Bearkats,JVILLE @ SHSU,-6.5,36.5,21 - 15
3,5,09/29 07:00 PM (Fri),N,NC State Wolfpack,Louisville Cardinals,Louisville Cardinals,NC State Wolfpack,LVILLE @ NC ST,-3.0,55.0,29 - 26
4,5,09/29 09:00 PM (Fri),N,UTEP Miners,Louisiana Tech Bulldogs,UTEP Miners,Louisiana Tech Bulldogs,UTEP vs LT,-1.0,53.0,27 - 26


In [4]:
check_team = 'Clemson'

short_df[(short_df['Favorite'].str.contains(check_team)) | (short_df['Underdog'].str.contains(check_team))]

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Description,Spread,Total,Implied Score
13,5,09/30 12:00 PM (Sat),N,Syracuse Orange,Clemson Tigers,Clemson Tigers,Syracuse Orange,CLEM @ SYR,-7.0,53.5,30 - 23


In [5]:
short_df[short_df['Mandatory'] == 'Y']

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Spread,Total,Description,Implied Score


In [6]:
team_list = [
    'UCLA',
    'Alabama Crimson',
    'Oregon',
    'Notre Dame',
    'Penn State',
    'SMU'
]

all_team_list = short_df['Favorite'].tolist() + short_df['Underdog'].tolist()
mandatory_list = []
for team in team_list:
    for t in all_team_list:
        if team in t:
            mandatory_list.append(t)

mandatory_picks = short_df[(short_df['Favorite'].isin(mandatory_list)) |
                           (short_df['Underdog'].isin(mandatory_list))]
mandatory_picks

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Spread,Total,Description,Implied Score
13,4,09/23 12:00 PM (Sat),N,TCU Horned Frogs,SMU Mustangs,TCU Horned Frogs,SMU Mustangs,-6.5,64.0,TCU vs SMU,35 - 28
17,4,09/23 03:30 PM (Sat),N,Alabama Crimson Tide,Ole Miss Rebels,Alabama Crimson Tide,Ole Miss Rebels,-7.0,57.0,BAMA vs Ole Miss,32 - 25
21,4,09/23 03:30 PM (Sat),N,Oregon Ducks,Colorado Buffaloes,Oregon Ducks,Colorado Buffaloes,-21.0,71.0,ORE vs COLO,46 - 25
29,4,09/23 03:30 PM (Sat),N,Utah Utes,UCLA Bruins,Utah Utes,UCLA Bruins,-4.5,51.0,UTAH vs UCLA,27 - 23
37,4,09/23 07:00 PM (Sat),N,Washington State Cougars,Oregon State Beavers,Oregon State Beavers,Washington State Cougars,-3.0,56.0,OSU @ WSU,29 - 26
47,4,09/23 07:30 PM (Sat),N,Notre Dame Fighting Irish,Ohio State Buckeyes,Ohio State Buckeyes,Notre Dame Fighting Irish,-3.5,54.5,tOSU @ ND,29 - 25
54,4,09/23 07:30 PM (Sat),N,Penn State Nittany Lions,Iowa Hawkeyes,Penn State Nittany Lions,Iowa Hawkeyes,-14.5,40.0,PSU vs IOWA,27 - 12


In [7]:
short_df.loc[mandatory_picks.index.tolist(), 'Mandatory'] = 'Y'
short_df[short_df['Mandatory'] == 'Y']

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Spread,Total,Description,Implied Score
13,4,09/23 12:00 PM (Sat),Y,TCU Horned Frogs,SMU Mustangs,TCU Horned Frogs,SMU Mustangs,-6.5,64.0,TCU vs SMU,35 - 28
17,4,09/23 03:30 PM (Sat),Y,Alabama Crimson Tide,Ole Miss Rebels,Alabama Crimson Tide,Ole Miss Rebels,-7.0,57.0,BAMA vs Ole Miss,32 - 25
21,4,09/23 03:30 PM (Sat),Y,Oregon Ducks,Colorado Buffaloes,Oregon Ducks,Colorado Buffaloes,-21.0,71.0,ORE vs COLO,46 - 25
29,4,09/23 03:30 PM (Sat),Y,Utah Utes,UCLA Bruins,Utah Utes,UCLA Bruins,-4.5,51.0,UTAH vs UCLA,27 - 23
37,4,09/23 07:00 PM (Sat),Y,Washington State Cougars,Oregon State Beavers,Oregon State Beavers,Washington State Cougars,-3.0,56.0,OSU @ WSU,29 - 26
47,4,09/23 07:30 PM (Sat),Y,Notre Dame Fighting Irish,Ohio State Buckeyes,Ohio State Buckeyes,Notre Dame Fighting Irish,-3.5,54.5,tOSU @ ND,29 - 25
54,4,09/23 07:30 PM (Sat),Y,Penn State Nittany Lions,Iowa Hawkeyes,Penn State Nittany Lions,Iowa Hawkeyes,-14.5,40.0,PSU vs IOWA,27 - 12


In [8]:
update_google_sheet(short_df)

11:01:30 INFO:Using credentials directory '/Users/hoke/dev/football/credentials'
11:01:30 INFO:Obtained credentails from existing 'token.json' file
11:01:30 INFO:Writing data for Hoke
11:01:30 INFO:file_cache is only supported with oauth2client<4.0.0
11:01:30 DEBUG:Attempting to write 63 rows to Hoke!A:J
11:01:30 DEBUG:URL being requested: POST https://sheets.googleapis.com/v4/spreadsheets/1bxqOme4RHBUMgc5ojKr-RgGt3vfetGDAZ1sfs0SeWw4/values/Hoke%21A%3AJ:append?valueInputOption=USER_ENTERED&alt=json
11:01:31 INFO:Appended 567 cells to Hoke!A:J
11:01:31 INFO:Writing data for Lou
11:01:31 INFO:file_cache is only supported with oauth2client<4.0.0
11:01:31 DEBUG:Attempting to write 63 rows to Lou!A:J
11:01:31 DEBUG:URL being requested: POST https://sheets.googleapis.com/v4/spreadsheets/1bxqOme4RHBUMgc5ojKr-RgGt3vfetGDAZ1sfs0SeWw4/values/Lou%21A%3AJ:append?valueInputOption=USER_ENTERED&alt=json
11:01:31 INFO:Appended 567 cells to Lou!A:J
11:01:31 INFO:Writing data for Tom
11:01:31 INFO:file_